## **Lab 2: KNN**

In [7]:
import pandas as pd

### 1. Dataset


In [8]:
fashion_data = pd.read_csv("fashion-mnist_test.csv")
X_fashion = fashion_data.iloc[:, 1:].values
y_fashion = fashion_data.iloc[:, 0].values


### 2. Model

In [ ]:
class KNN:
    def __init__(self):